In [ ]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.6 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
import keras_tuner as kt
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
from google.colab import files
import pandas as pd

# Upload CSV
uploaded = files.upload()

Saving Dogecoin_cleaned.csv to Dogecoin_cleaned.csv


In [ ]:
# Read the CSV (assuming only one file is uploaded)
filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)



In [ ]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='mixed', dayfirst=True, errors='coerce')



In [ ]:
df.shape

(707, 25)

In [ ]:
crypto = "Dogecoin"

In [ ]:
# Selecting relevant columns for LSTM
features = ['Open', 'High', 'Low', 'Volume', 'VWAP', 'Volatility', 'Price_Change', 'RSI', 'MACD',
            'MA_7', 'MA_14', 'EMA_7', 'EMA_14', 'Rolling_Std_7', 'Rolling_Std_14',
            'Day_of_Week', 'Month', 'Week_of_Year', 'Close_Lag_1', 'Close_Lag_2', 'Close_Lag_3']
target = 'Close'

In [ ]:
crypto_df = df[df['Crypto'] == crypto].drop(columns=['Crypto']).dropna()

In [ ]:
df.isnull().sum()

,0
Timestamp,0
Open,0
High,0
Low,0
Close,0
VWAP,0
Volume,0
Count,0
Crypto,0
Volatility,0


In [ ]:
df[['Close_Lag_1', 'Close_Lag_2', 'Close_Lag_3']].ffill(inplace=True)
df[['Rolling_Std_7', 'Rolling_Std_14']].interpolate(method='linear', inplace=True)

print(df.isnull().sum())


Timestamp         0
Open              0
High              0
Low               0
Close             0
VWAP              0
Volume            0
Count             0
Crypto            0
Volatility        0
Price_Change      0
RSI               0
MA_7              0
MA_14             0
Close_Lag_1       0
Close_Lag_2       0
Close_Lag_3       0
Rolling_Std_7     0
Rolling_Std_14    0
EMA_7             0
EMA_14            0
MACD              0
Day_of_Week       0
Month             0
Week_of_Year      0
dtype: int64


<ipython-input-11-5ec87b0f3c38>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[['Close_Lag_1', 'Close_Lag_2', 'Close_Lag_3']] = df[['Close_Lag_1', 'Close_Lag_2', 'Close_Lag_3']].fillna(method='ffill')


In [ ]:
# Drop remaining rows with missing values and verify
df.dropna(inplace=True)

# ✅ Confirm no missing values remain
assert df.isnull().sum().sum() == 0, "There are still missing values!"

# ✅ Display dataset size after dropping missing values
print(f"Dataset size after dropping missing values: {df.shape}")



Timestamp         0
Open              0
High              0
Low               0
Close             0
VWAP              0
Volume            0
Count             0
Crypto            0
Volatility        0
Price_Change      0
RSI               0
MA_7              0
MA_14             0
Close_Lag_1       0
Close_Lag_2       0
Close_Lag_3       0
Rolling_Std_7     0
Rolling_Std_14    0
EMA_7             0
EMA_14            0
MACD              0
Day_of_Week       0
Month             0
Week_of_Year      0
dtype: int64
Dataset size after dropping missing values: (707, 25)


In [ ]:
df.head()

,Timestamp,Open,High,Low,Close,VWAP,Volume,Count,Crypto,Volatility,...,Close_Lag_2,Close_Lag_3,Rolling_Std_7,Rolling_Std_14,EMA_7,EMA_14,MACD,Day_of_Week,Month,Week_of_Year
0,2023-08-03,0.074056,0.074247,0.070000,0.071051,0.072186,43200000.0,3226,Dogecoin,0.004247,...,0.074764,0.074779,0.002879,0.003986,0.075029,0.077503,-0.002473,2,3,10
1,2023-09-03,0.071146,0.072236,0.064001,0.065999,0.067470,43500000.0,4681,Dogecoin,0.008235,...,0.074056,0.074764,0.003594,0.004856,0.072772,0.075969,-0.003197,3,3,10
2,2023-10-03,0.065984,0.066205,0.062617,0.065945,0.064740,46200000.0,4255,Dogecoin,0.003588,...,0.071051,0.074056,0.004066,0.005568,0.071065,0.074632,-0.003567,4,3,10
3,2023-11-03,0.065944,0.068865,0.062785,0.066603,0.065701,38800000.0,3890,Dogecoin,0.006079,...,0.065999,0.071051,0.004196,0.005935,0.069949,0.073562,-0.003612,5,3,10
4,2023-12-03,0.066558,0.071450,0.065882,0.070962,0.067507,51300000.0,3204,Dogecoin,0.005568,...,0.065945,0.065999,0.003767,0.005657,0.070203,0.073215,-0.003013,6,3,10


In [ ]:
df = df.drop_duplicates(subset=['Timestamp'], keep='first')

# Sort data by Timestamp
df = df.sort_values(by=['Timestamp'])

In [ ]:
# Save cleaned dataset
cleaned_csv_path = "C:\\Users\\annae\\OneDrive\\Documents\\cryptointel\\cleaned_dogecoin_csv.csv"
df.to_csv(cleaned_csv_path, index=False)

# Confirm file saved
print(f"Cleaned dataset saved at: {cleaned_csv_path}")

# Confirm file saved
print(f"Cleaned dataset saved at: {cleaned_csv_path}")

Cleaned dataset saved at: C:\Users\annae\OneDrive\Documents\cryptointel\cleaned_dogecoin_csv.csv
Cleaned dataset saved at: C:\Users\annae\OneDrive\Documents\cryptointel\cleaned_dogecoin_csv.csv


In [ ]:
df = pd.read_csv("C:\\Users\\annae\\OneDrive\\Documents\\cryptointel\\cleaned_dogecoin_csv.csv")

In [ ]:
import plotly.express as px

# Create line plot
fig = px.line(df, x='Timestamp', y='Close', labels={'Timestamp': 'Date', 'Close': 'Close Price'})

# Update trace settings
fig.update_traces(
    line=dict(width=2, color='orange'),
    opacity=0.8
)

# Update layout with title and styling
fig.update_layout(
    title='Whole Period of Dogecoin Close Price',
    plot_bgcolor='white',
    font=dict(size=15, color='black'),
    xaxis_showgrid=False,
    yaxis_showgrid=False
)

# Show plot
fig.show()



In [1]:
# Use separate scalers for features & target
scaler_features, scaler_target = MinMaxScaler(), MinMaxScaler()

crypto_scaled_features = scaler_features.fit_transform(crypto_df[features])
target_scaled = scaler_target.fit_transform(crypto_df[[target]])

# Combine after scaling
crypto_scaled = pd.DataFrame(
    np.hstack((crypto_scaled_features, target_scaled)),
    columns=features + [target],
    index=crypto_df.index
)

# Train-Test Split
train_size = int(len(crypto_scaled) * 0.8)
train, test = crypto_scaled.iloc[:train_size], crypto_scaled.iloc[train_size:]

# ✅ Increase sequence length to 60-120
def create_sequences(data, seq_length):
    x_data = [data.iloc[i-seq_length:i][features].values for i in range(seq_length, len(data))]
    y_data = [data.iloc[i][target] for i in range(seq_length, len(data))]
    return np.array(x_data), np.array(y_data)

# Define Model for Keras Tuner
def build_model(hp):
    sequence_length = hp.Int('sequence_length', min_value=5, max_value=20, step=5)

    model = Sequential([
        Input(shape=(sequence_length, len(features))),
        Bidirectional(LSTM(hp.Int('units_1', 128, 768, 128), return_sequences=True)),
        Dropout(hp.Float('dropout_1', 0.005, 0.02, 0.005)),  # ✅ Reduce Dropout

        LSTM(hp.Int('units_2', 128, 768, 128), return_sequences=True),
        Dropout(hp.Float('dropout_2', 0.005, 0.02, 0.005)),

        LSTM(hp.Int('units_3', 128, 768, 128), return_sequences=False),
        Dropout(hp.Float('dropout_3', 0.005, 0.02, 0.005)),

        Dense(hp.Int('dense_units', 128, 512, 128), activation='tanh'),
        Dense(1, activation='linear')
    ])

    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', [0.0003, 0.0005, 0.001])),
        loss=Huber(delta=100)  # ✅ Larger delta for Bitcoin
    )
    return model

# ✅ Increase trials for better tuning
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=30,
    executions_per_trial=1,
    directory='lstm_tuning',
    project_name=f'crypto_forecasting_{crypto}',
    overwrite=True
)

# Run Hyperparameter Tuning
x_train, y_train = create_sequences(train, 60)
x_test, y_test = create_sequences(test, 60)

tuner.search(x_train, y_train, validation_data=(x_test, y_test), epochs=40, batch_size=64, verbose=1)

# Get Best Hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_sequence_length = best_hps.get('sequence_length')

# Train Final Model with Best Sequence Length
x_train, y_train = create_sequences(train, optimal_sequence_length)
x_test, y_test = create_sequences(test, optimal_sequence_length)

model = build_model(best_hps)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

model.fit(
    x_train, y_train, validation_data=(x_test, y_test),
    epochs=80, batch_size=64, verbose=1, callbacks=[early_stopping]
)

# ✅ Save Model
model_path = os.path.join("models", f"LSTM_{crypto}.h5")
model.save(model_path)
print(f"✅ Model saved: {model_path}")

# Make Predictions
y_pred = model.predict(x_test)

# ✅ Fix inverse transformation issue
y_pred_rescaled = scaler_target.inverse_transform(y_pred.reshape(-1, 1))[:, 0]
y_test_rescaled = scaler_target.inverse_transform(y_test.reshape(-1, 1))[:, 0]

# ✅ Plot Predictions
plt.figure(figsize=(12, 6))
plt.plot(test.index[optimal_sequence_length:], y_test_rescaled, label="Actual", color="blue")
plt.plot(test.index[optimal_sequence_length:], y_pred_rescaled, label="LSTM Predicted", color="orange")
plt.title(f"LSTM Model - Actual vs Predicted for {crypto}")
plt.legend()
plt.show()

# ✅ Compute Performance Metrics
performance_metrics = {
    "MAE": mean_absolute_error(y_test_rescaled, y_pred_rescaled),
    "MSE": mean_squared_error(y_test_rescaled, y_pred_rescaled),
    "RMSE": np.sqrt(mean_squared_error(y_test_rescaled, y_pred_rescaled)),
    "R-squared": r2_score(y_test_rescaled, y_pred_rescaled)
}

# ✅ Print & Save Metrics
metrics_path = f"LSTM_Performance_{crypto}.csv"
pd.DataFrame([performance_metrics]).to_csv(metrics_path, index=False)
print(f"📊 Performance for {crypto}: {performance_metrics}")

In [ ]:
import pandas as pd

# ✅ Load the Bitcoin dataset
bitcoin_data_path = r"C:C:\\Users\\annae\\OneDrive\\Documents\\cryptointel\\cleaned_dogecoin_data_csv.csv"
bitcoin_df = pd.read_csv(bitcoin_data_path, parse_dates=['Timestamp'], dayfirst=True, index_col='Timestamp')

# ✅ Define target variable (Close price)
close_prices = bitcoin_df['Close']

# ✅ Define exogenous features
features = ['MACD', 'EMA_7', 'Close_Lag_1']
exog_features = bitcoin_df[features]

# ✅ Display first few rows
print(bitcoin_df.head())


In [ ]:
from statsmodels.tsa.stattools import adfuller

# Perform ADF Test on Close Price
adf_stat, p_value, _, _, critical_values, _ = adfuller(close_prices)

# Print ADF Test Results
print(f"ADF Statistic: {adf_stat:.4f}")
print(f"p-value: {p_value:.4f}")
print("Critical Values:")
for key, value in critical_values.items():
    print(f"  {key}: {value:.4f}")

# Check stationarity
if p_value < 0.05:
    print(" Data is stationary, ARIMA can be applied directly (d=0).")
else:
    print(" Data is non-stationary, differencing is required (d>0).")


In [ ]:
# ✅ Apply first-order differencing if needed
diff_close_prices = close_prices.diff().dropna()

# ✅ Perform ADF test after differencing
adf_stat_diff, p_value_diff, _, _, critical_values_diff, _ = adfuller(diff_close_prices)

# ✅ Print ADF test results after differencing
print(f"After Differencing - ADF Statistic: {adf_stat_diff:.4f}")
print(f"After Differencing - p-value: {p_value_diff:.4f}")
print("Critical Values:")
for key, value in critical_values_diff.items():
    print(f"  {key}: {value:.4f}")

# ✅ Check stationarity after differencing
if p_value_diff < 0.05:
    print("✅ Data is now stationary, ARIMA can be applied with d=1.")
else:
    print("⚠️ Still non-stationary, consider applying second-order differencing (d=2).")


In [ ]:
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# ✅ Plot ACF & PACF
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# ✅ ACF Plot (for identifying q)
plot_acf(diff_close_prices, ax=axes[0], lags=40)  # Limit lags for better visibility
axes[0].set_title("Autocorrelation Function (ACF)")

# ✅ PACF Plot (for identifying p)
plot_pacf(diff_close_prices, ax=axes[1], lags=40, method="ywm")  # Yule-Walker method for stability
axes[1].set_title("Partial Autocorrelation Function (PACF)")

plt.tight_layout()
plt.show()


In [ ]:
import itertools
import warnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error

# ✅ Define possible values for p, d, q, P, D, Q
p = range(0, 3)
d = range(0, 2)
q = range(0, 3)
P = range(0, 2)
D = range(0, 2)
Q = range(0, 2)
s = [7]  # Weekly seasonality

# ✅ Initialize tracking variables
best_mae = float("inf")
best_params = None
best_model = None

# ✅ Suppress warnings for failed models
warnings.filterwarnings("ignore")

# ✅ Grid search over all parameter combinations
for order in itertools.product(p, d, q):
    for seasonal_order in itertools.product(P, D, Q, s):
        try:
            model = SARIMAX(
                close_prices,
                exog=exog_features,
                order=order,
                seasonal_order=seasonal_order
            )
            result = model.fit(disp=False)

            # ✅


In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# ✅ Define the final optimized SARIMAX model
final_order = (0, 0, 0)
final_seasonal_order = (0, 0, 1, 7)

# ✅ Train the model
sarimax_final = SARIMAX(
    close_prices,
    exog=exog_features,
    order=final_order,
    seasonal_order=final_seasonal_order
)

sarimax_result_final = sarimax_final.fit(disp=False)  # Suppress unnecessary output

# ✅ Print summary of the final model
print(sarimax_result_final.summary())


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# ✅ Forecast the next 30 days
sarimax_forecast = sarimax_result_final.forecast(steps=30, exog=exog_features.iloc[-30:])

# ✅ Compute performance metrics
mae_sarimax = mean_absolute_error(close_prices[-30:], sarimax_forecast)
mse_sarimax = mean_squared_error(close_prices[-30:], sarimax_forecast)
rmse_sarimax = np.sqrt(mse_sarimax)
r2_sarimax = r2_score(close_prices[-30:], sarimax_forecast)

# ✅ Collect performance metrics in a dictionary
sarimax_performance = {
    "MAE": mae_sarimax,
    "MSE": mse_sarimax,
    "RMSE": rmse_sarimax,
    "R-squared": r2_sarimax
}

# ✅ Print final performance metrics
print(f"📊 Final SARIMAX Performance Metrics:\n{sarimax_performance}")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

# ✅ Load the trained LSTM model
lstm_model_path = "C:\\Users\\annae\\OneDrive\\Documents\\cryptointel\\LSTM_dogecoin.h5"
lstm_model = load_model(lstm_model_path)

# ✅ Normalize features before feeding into the LSTM model
features = ['Open', 'High', 'Low', 'Volume', 'VWAP', 'Volatility', 'Price_Change', 'RSI', 'MACD',
            'MA_7', 'MA_14', 'EMA_7', 'EMA_14', 'Rolling_Std_7', 'Rolling_Std_14',
            'Day_of_Week', 'Month', 'Week_of_Year', 'Close_Lag_1', 'Close_Lag_2', 'Close_Lag_3']
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(bitcoin_df[features])

# ✅ Prepare test data for LSTM
look_back = 5
test_scaled = scaled_data[-30:]  # Last 30 days for testing
X_test = []

# ✅ Create input sequences for LSTM (with look-back window)
for i in range(look_back, len(test_scaled)):
    X_test.append(test_scaled[i - look_back:i, :])

X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], look_back, len(features)))

# ✅ LSTM Predictions
lstm_predictions = lstm_model.predict(X_test)

# ✅ Inverse transform LSTM predictions to original scale
lstm_predictions = scaler.inverse_transform(
    np.concatenate([np.zeros((lstm_predictions.shape[0], len(features)-1)), lstm_predictions], axis=1)
)[:, -1]  # Extract only Close Price

# ✅ Generate future dates for plotting (30 days ahead)
final_future_dates = pd.date_range(start=close_prices.index[-1], periods=30 + 1, freq='D')[1:]

# ✅ Adjust the test index for LSTM predictions
lstm_test_index = final_future_dates[look_back:]

# ✅ Plot SARIMAX vs. LSTM vs. Actual Prices
plt.figure(figsize=(12, 6))
plt.plot(close_prices.index, close_prices, label="Actual Prices", color='blue')
plt.plot(final_future_dates, sarimax_forecast, label="Final SARIMAX Forecast", color='orange')
plt.plot(lstm_test_index, lstm_predictions, label="LSTM Predictions", color='green')

plt.xlabel('Date')
plt.ylabel('Dogecoin Price (USD)')
plt.title("Comparison: SARIMAX vs. LSTM vs. Actual Prices")
plt.legend()
plt.grid(True)
plt.show()
